# day 24

https://adventofcode.com/2018/day/24

In [216]:
import os
import re

import eri.logging as logging

In [217]:
FNAME = os.path.join('data', 'day24.txt')

LOGGER = logging.getLogger('day24')
logging.configure()

## part 1

### problem statement:

> After a weird buzzing noise, you appear back at the man's cottage. He seems relieved to see his friend, but quickly notices that the little reindeer caught some kind of cold while out exploring.
> 
> The portly man explains that this reindeer's immune system isn't similar to regular reindeer immune systems:
> 
> The immune system and the infection each have an army made up of several groups; each group consists of one or more identical units. The armies repeatedly fight until only one army has units remaining.
> 
> Units within a group all have the same hit points (amount of damage a unit can take before it is destroyed), attack damage (the amount of damage each unit deals), an attack type, an initiative (higher initiative units attack first and win ties), and sometimes weaknesses or immunities. Here is an example group:
> 
>     18 units each with 729 hit points (weak to fire; immune to cold, slashing)
>      with an attack that does 8 radiation damage at initiative 10
> 
> Each group also has an effective power: the number of units in that group multiplied by their attack damage. The above group has an effective power of 18 * 8 = 144. Groups never have zero or negative units; instead, the group is removed from combat.
> 
> Each fight consists of two phases: target selection and attacking.
> 
> During the target selection phase, each group attempts to choose one target. In decreasing order of effective power, groups choose their targets; in a tie, the group with the higher initiative chooses first. The attacking group chooses to target the group in the enemy army to which it would deal the most damage (after accounting for weaknesses and immunities, but not accounting for whether the defending group has enough units to actually receive all of that damage).
> 
> If an attacking group is considering two defending groups to which it would deal equal damage, it chooses to target the defending group with the largest effective power; if there is still a tie, it chooses the defending group with the highest initiative. If it cannot deal any defending groups damage, it does not choose a target. Defending groups can only be chosen as a target by one attacking group.
> 
> At the end of the target selection phase, each group has selected zero or one groups to attack, and each group is being attacked by zero or one groups.
> 
> During the attacking phase, each group deals damage to the target it selected, if any. Groups attack in decreasing order of initiative, regardless of whether they are part of the infection or the immune system. (If a group contains no units, it cannot attack.)
> 
> The damage an attacking group deals to a defending group depends on the attacking group's attack type and the defending group's immunities and weaknesses. By default, an attacking group would deal damage equal to its effective power to the defending group. However, if the defending group is immune to the attacking group's attack type, the defending group instead takes no damage; if the defending group is weak to the attacking group's attack type, the defending group instead takes double damage.
> 
> The defending group only loses whole units from damage; damage is always dealt in such a way that it kills the most units possible, and any remaining damage to a unit that does not immediately kill it is ignored. For example, if a defending group contains 10 units with 10 hit points each and receives 75 damage, it loses exactly 7 units and is left with 3 units at full health.
> 
> After the fight is over, if both armies still contain units, a new fight begins; combat only ends once one army has lost all of its units.
> 
> For example, consider the following armies:
> 
>     Immune System:
>     17 units each with 5390 hit points (weak to radiation, bludgeoning) with
>      an attack that does 4507 fire damage at initiative 2
>     989 units each with 1274 hit points (immune to fire; weak to bludgeoning,
>      slashing) with an attack that does 25 slashing damage at initiative 3
>     
>     Infection:
>     801 units each with 4706 hit points (weak to radiation) with an attack
>      that does 116 bludgeoning damage at initiative 1
>     4485 units each with 2961 hit points (immune to radiation; weak to fire,
>      cold) with an attack that does 12 slashing damage at initiative 4
>
> If these armies were to enter combat, the following fights, including details during the target selection and attacking phases, would take place:
> 
>     Immune System:
>     Group 1 contains 17 units
>     Group 2 contains 989 units
>     Infection:
>     Group 1 contains 801 units
>     Group 2 contains 4485 units
>     
>     Infection group 1 would deal defending group 1 185832 damage
>     Infection group 1 would deal defending group 2 185832 damage
>     Infection group 2 would deal defending group 2 107640 damage
>     Immune System group 1 would deal defending group 1 76619 damage
>     Immune System group 1 would deal defending group 2 153238 damage
>     Immune System group 2 would deal defending group 1 24725 damage
>     
>     Infection group 2 attacks defending group 2, killing 84 units
>     Immune System group 2 attacks defending group 1, killing 4 units
>     Immune System group 1 attacks defending group 2, killing 51 units
>     Infection group 1 attacks defending group 1, killing 17 units
>
>     Immune System:
>     Group 2 contains 905 units
>     Infection:
>     Group 1 contains 797 units
>     Group 2 contains 4434 units
>     
>     Infection group 1 would deal defending group 2 184904 damage
>     Immune System group 2 would deal defending group 1 22625 damage
>     Immune System group 2 would deal defending group 2 22625 damage
>     
>     Immune System group 2 attacks defending group 1, killing 4 units
>     Infection group 1 attacks defending group 2, killing 144 units
>
>     Immune System:
>     Group 2 contains 761 units
>     Infection:
>     Group 1 contains 793 units
>     Group 2 contains 4434 units
>     
>     Infection group 1 would deal defending group 2 183976 damage
>     Immune System group 2 would deal defending group 1 19025 damage
>     Immune System group 2 would deal defending group 2 19025 damage
>     
>     Immune System group 2 attacks defending group 1, killing 4 units
>     Infection group 1 attacks defending group 2, killing 143 units
>
>     Immune System:
>     Group 2 contains 618 units
>     Infection:
>     Group 1 contains 789 units
>     Group 2 contains 4434 units
>     
>     Infection group 1 would deal defending group 2 183048 damage
>     Immune System group 2 would deal defending group 1 15450 damage
>     Immune System group 2 would deal defending group 2 15450 damage
>     
>     Immune System group 2 attacks defending group 1, killing 3 units
>     Infection group 1 attacks defending group 2, killing 143 units
>
>     Immune System:
>     Group 2 contains 475 units
>     Infection:
>     Group 1 contains 786 units
>     Group 2 contains 4434 units
>     
>     Infection group 1 would deal defending group 2 182352 damage
>     Immune System group 2 would deal defending group 1 11875 damage
>     Immune System group 2 would deal defending group 2 11875 damage
>     
>     Immune System group 2 attacks defending group 1, killing 2 units
>     Infection group 1 attacks defending group 2, killing 142 units
>
>     Immune System:
>     Group 2 contains 333 units
>     Infection:
>     Group 1 contains 784 units
>     Group 2 contains 4434 units
>     
>     Infection group 1 would deal defending group 2 181888 damage
>     Immune System group 2 would deal defending group 1 8325 damage
>     Immune System group 2 would deal defending group 2 8325 damage
>     
>     Immune System group 2 attacks defending group 1, killing 1 unit
>     Infection group 1 attacks defending group 2, killing 142 units
>
>     Immune System:
>     Group 2 contains 191 units
>     Infection:
>     Group 1 contains 783 units
>     Group 2 contains 4434 units
>     
>     Infection group 1 would deal defending group 2 181656 damage
>     Immune System group 2 would deal defending group 1 4775 damage
>     Immune System group 2 would deal defending group 2 4775 damage
>     
>     Immune System group 2 attacks defending group 1, killing 1 unit
>     Infection group 1 attacks defending group 2, killing 142 units
>
>     Immune System:
>     Group 2 contains 49 units
>     Infection:
>     Group 1 contains 782 units
>     Group 2 contains 4434 units
>     
>     Infection group 1 would deal defending group 2 181424 damage
>     Immune System group 2 would deal defending group 1 1225 damage
>     Immune System group 2 would deal defending group 2 1225 damage
>     
>     Immune System group 2 attacks defending group 1, killing 0 units
>     Infection group 1 attacks defending group 2, killing 49 units
>
>     Immune System:
>     No groups remain.
>     Infection:
>     Group 1 contains 782 units
>     Group 2 contains 4434 units
>
> In the example above, the winning army ends up with 782 + 4434 = 5216 units.
> 
> You scan the reindeer's condition (your puzzle input); the white-bearded man looks nervous. As it stands now, how many units would the winning army have?

#### loading data

In [218]:
def load_data(fname=FNAME):
    with open(fname) as fp:
        return fp.read().strip()

#### function def

In [1020]:
class Army(object):
    """collection of groups"""
    def __init__(self, type):
        self.groups = []
        self.type = type
    
    def __repr__(self):
        return '<Army type={} n={} n_living={}>'.format(
            self.type, self.num_units, self.num_living_units
        )
    
    def add_group(self, g):
        self.groups.append(g)
    
    def init_from_summary(self, s, immune_boost=0):
        LOGGER.debug('initalizing army from summary block')
        for (i, line) in enumerate(s, start=1):
            g = Group(i, self.type)
            g.init_from_summary(line, immune_boost)
            self.add_group(g)
    
    @property
    def is_dead(self):
        return all(g.is_dead for g in self.groups)
    
    @property
    def is_alive(self):
        return not self.is_dead()
    
    @property
    def living_groups(self):
        """return the groups which have living units still"""
        return [g for g in self.groups if g.is_alive]
    
    @property
    def num_units(self):
        return sum(len(g.units) for g in self.groups)
    
    @property
    def num_living_units(self):
        return sum(len(g.living_units) for g in self.living_groups)
    
    def choose_targets(self, tgts):
        # reset all targets first
        for g in self.groups:
            g.target = None
            
        attacking = sorted(
            self.living_groups,
            key=lambda g: (-g.effective_power, -g.initiative)
        )
        LOGGER.debug('target choosing order is {}'.format(attacking))
        to_attack = set(tgts)
        for g in attacking:
            g.choose_target(to_attack)
            if g.target is not None:
                to_attack.remove(g.target)
            if to_attack == set():
                break

In [1021]:
class Group(object):
    """collection of units"""
    def __init__(self, id=None, type=None):
        self.units = []
        self.id = id
        self.type = type
        self.target = None
    
    def __repr__(self):
        return '<Group {} {} n0={} ep={} init={} nc={}>'.format(
            self.type, self.id, len(self.units), self.effective_power, self.initiative,
            len(self.living_units)
        )
    
    def add_unit(self, hp, attack, attack_type, initiative,
                 weaknesses=None, immunities=None):
        self.units.append(
            Unit(
                hp, attack, attack_type, initiative,
                weaknesses, immunities
            )
        )
    
    @property
    def living_units(self):
        return [u for u in self.units if u.is_alive]
    
    def init_from_summary(self, s, immune_boost=0):
        LOGGER.debug('parsing summary line: {}'.format(s))
        n = int(re.match('^(\d+)', s).groups()[0])
        hp = int(re.search('with (\d+) hit', s).groups()[0])
        attack = int(re.search('that does (\d+)', s).groups()[0]) + immune_boost
        attack_type = re.search('does \d+ (\w+) damage', s).groups()[0]
        initiative = int(re.search('at initiative (\d+)', s).groups()[0])

        try:
            immunities = re.search('immune to ([\w, ]+)[;\)]', s).groups()[0]
            immunities = set(immunities.split(', '))
        except:
            immunities = None
            LOGGER.debug('record has no immunities')

        try:
            weaknesses = re.search('weak to ([\w, ]+)[;\)]', s).groups()[0]
            weaknesses = set(weaknesses.split(', '))
        except:
            weaknesses = None
            LOGGER.debug('record has no weaknesses')

        for i in range(n):
            self.add_unit(hp=hp, attack=attack, attack_type=attack_type,
                          initiative=initiative, weaknesses=weaknesses, 
                          immunities=immunities)
    
    @property
    def effective_power(self):
        return len(self.living_units) * self.units[0].attack
    
    def choose_target(self, groups):
        LOGGER.debug('attempting to choose target for {} out of {}'.format(self, groups))
        tgt = max(
            groups,
            key=lambda g2: (
                self.possible_damage(g2),
                g2.effective_power,
                g2.initiative
            )
        )
        # it's possible we just chose a target to whom we can't
        # do any damage; then do nothing
        if self.possible_damage(tgt) == 0:
            LOGGER.debug('group {} cannot do damage to any other group'.format(self))
            return
        
        # we picked a target, we're good
        LOGGER.debug('group {} will target {}'.format(self, tgt))
        self.target = tgt
    
    def possible_damage(self, g):
        """how much damage would be done to a different group g"""
        if self.attack_type in g.weaknesses:
            d = 2 * self.effective_power
        elif self.attack_type in g.immunities:
            d = 0
        else:
            d = self.effective_power
        LOGGER.debug('group {} would do {} damage to {}'.format(self, d, g))
        return d
    
    def attack(self):
        if self.target is None:
            LOGGER.debug('no target, skipping attack')
        else:
            LOGGER.debug('attacking target {}'.format(self.target))
            self.target.take_damage(power=self.effective_power, attack_type=self.attack_type)
        
    def take_damage(self, power, attack_type):
        """this group object is *being* attacked; assign whole-unit-damage"""
        damage_left = power
        if attack_type in self.weaknesses:
            damage_left *= 2
        elif attack_type in self.immunities:
            damage_left = 0
        
        LOGGER.debug('{} damage remaining'.format(damage_left))
        # order units by remaining hp, kill them one by one
        ordered_units = sorted(self.living_units, key=lambda u: u.hp)
        num_killed = 0
        for u in ordered_units:
            if damage_left >= u.hp:
                LOGGER.debug('killing unit {}'.format(u))
                damage_left -= u.hp
                LOGGER.debug('{} damage remaining'.format(damage_left))
                u.hp = 0
                num_killed += 1
            else:
                LOGGER.debug('insufficient damage to kill unit, group damage is done')
                break
        LOGGER.debug('group {} just lost {} units'.format(self, num_killed))
    
    @property
    def is_dead(self):
        return all(u.is_dead for u in self.units)
    
    @property
    def is_alive(self):
        return not self.is_dead
    
    @property
    def initiative(self):
        return self.units[0].initiative
    
    @property
    def attack_type(self):
        return self.units[0].attack_type
    
    @property
    def weaknesses(self):
        return self.units[0].weaknesses
    
    @property
    def immunities(self):
        return self.units[0].immunities

In [1022]:
class Unit(object):
    def __init__(self, hp, attack, attack_type, initiative,
                 weaknesses=None, immunities=None):
        self.hp = hp
        self.attack = attack
        self.attack_type = attack_type
        self.initiative = initiative
        self.weaknesses = weaknesses or set()
        self.immunities = immunities or set()
    
    def __repr__(self):
        return '<Unit hp={} attack={} {} init={}>'.format(
            self.hp, self.attack, self.attack_type, self.initiative
        )
    
    @property
    def is_dead(self):
        return self.hp <= 0
    
    @property
    def is_alive(self):
        return not self.is_dead

In [1092]:
class Battle(object):
    def __init__(self, battle_summary, immune_boost=0):
        self.battle_summary = battle_summary
        self.immune_boost = immune_boost
        self.stalemate = False
        self.parse_battle_summary()
        
    def parse_battle_summary(self):
        immune_summary, infection_summary = self.battle_summary.split('\n\n')
        immune_summary = immune_summary.split('\n')[1:]
        infection_summary = infection_summary.split('\n')[1:]
        
        self.immune_army = Army('immune')
        self.immune_army.init_from_summary(immune_summary, immune_boost=self.immune_boost)
        
        self.infection_army = Army('infection')
        self.infection_army.init_from_summary(infection_summary)
    
    def fight(self):
        self.choose_targets()
        self.attack()
    
    def choose_targets(self):
        self.immune_army.choose_targets(self.infection_army.living_groups)
        self.infection_army.choose_targets(self.immune_army.living_groups)
    
    def attack(self):
        infection_n_living_start = self.infection_army.num_living_units
        immune_n_living_start = self.immune_army.num_living_units
        
        attack_order = sorted(
            self.infection_army.living_groups + self.immune_army.living_groups,
            key=lambda g: -g.initiative
        )
        
        LOGGER.debug('attack order is {}'.format(attack_order))
        
        for g in attack_order:
            LOGGER.debug('group {} now attacks'.format(g))
            g.attack()
        
        num_infection_deaths = infection_n_living_start - self.infection_army.num_living_units
        num_immune_deaths = immune_n_living_start - self.immune_army.num_living_units
        if num_infection_deaths == 0 and num_immune_deaths == 0:
            LOGGER.info('we are in a loop')
            self.stalemate = True
        else:
            LOGGER.info('infection deaths: {}'.format(num_infection_deaths))
            LOGGER.info('immune deaths: {}'.format(num_immune_deaths))
    
    @property
    def over(self):
        return self.immune_army.is_dead or self.infection_army.is_dead or self.stalemate
    
    def run(self):
        while not self.over:
            self.fight()
            self.log()
    
    def log(self):
        LOGGER.info('immune: {:,}, infection: {:,}'.format(
            self.immune_army.num_living_units, self.infection_army.num_living_units
        ))
    
    @property
    def winner(self):
        if not self.over:
            return None
        elif self.stalemate:
            return 'stalemate'
        elif self.immune_army.is_dead and self.infection_army.is_dead:
            raise ValueError('both armies are dead...')
        elif self.immune_army.is_dead:
            return self.infection_army
        elif self.infection_army.is_dead:
            return self.immune_army
    
    @property
    def winning_score(self):
        return self.winner.num_living_units

In [1055]:
def q_1(data):
    b = Battle(data)
    b.run()
    return b.winning_score

#### tests

In [1056]:
test_s = ("18 units each with 729 hit points (weak to fire; immune to cold, slashing)"
          " with an attack that does 8 radiation damage at initiative 10")

In [1057]:
g = Group()
g.init_from_summary(test_s)

2019-01-11 23:59:40,248 DEBUG    [day24.init_from_summary:29] parsing summary line: 18 units each with 729 hit points (weak to fire; immune to cold, slashing) with an attack that does 8 radiation damage at initiative 10


In [1058]:
assert g.effective_power == 144

In [1059]:
assert g.immunities.intersection({'cold', 'slashing'}) == {'cold', 'slashing'}

In [1060]:
assert g.weaknesses.intersection({'fire'}) == {'fire'}

In [1061]:
test_data = """Immune System:
17 units each with 5390 hit points (weak to radiation, bludgeoning) with an attack that does 4507 fire damage at initiative 2
989 units each with 1274 hit points (immune to fire; weak to bludgeoning, slashing) with an attack that does 25 slashing damage at initiative 3

Infection:
801 units each with 4706 hit points (weak to radiation) with an attack that does 116 bludgeoning damage at initiative 1
4485 units each with 2961 hit points (immune to radiation; weak to fire, cold) with an attack that does 12 slashing damage at initiative 4")"""

In [1062]:
b = Battle(test_data)

2019-01-11 23:59:41,071 DEBUG    [day24.init_from_summary:16] initalizing army from summary block
2019-01-11 23:59:41,074 DEBUG    [day24.init_from_summary:29] parsing summary line: 17 units each with 5390 hit points (weak to radiation, bludgeoning) with an attack that does 4507 fire damage at initiative 2
2019-01-11 23:59:41,077 DEBUG    [day24.init_from_summary:41] record has no immunities
2019-01-11 23:59:41,080 DEBUG    [day24.init_from_summary:29] parsing summary line: 989 units each with 1274 hit points (immune to fire; weak to bludgeoning, slashing) with an attack that does 25 slashing damage at initiative 3
2019-01-11 23:59:41,087 DEBUG    [day24.init_from_summary:16] initalizing army from summary block
2019-01-11 23:59:41,089 DEBUG    [day24.init_from_summary:29] parsing summary line: 801 units each with 4706 hit points (weak to radiation) with an attack that does 116 bludgeoning damage at initiative 1
2019-01-11 23:59:41,092 DEBUG    [day24.init_from_summary:41] record has no

In [1063]:
b.infection_army.groups

[<Group infection 1 n0=801 ep=92916 init=1 nc=801>,
 <Group infection 2 n0=4485 ep=53820 init=4 nc=4485>]

In [1064]:
b.infection_army.groups[0].units[0]

<Unit hp=4706 attack=116 bludgeoning init=1>

In [1065]:
b.immune_army.groups

[<Group immune 1 n0=17 ep=76619 init=2 nc=17>,
 <Group immune 2 n0=989 ep=24725 init=3 nc=989>]

In [1066]:
b.immune_army.groups[0].units[0]

<Unit hp=5390 attack=4507 fire init=2>

In [1067]:
for g in b.immune_army.groups:
    print('group {} has weaknesses {} and immunities {}'.format(g, g.weaknesses, g.immunities))
    
for g in b.infection_army.groups:
    print('group {} has weaknesses {} and immunities {}'.format(g, g.weaknesses, g.immunities))

group <Group immune 1 n0=17 ep=76619 init=2 nc=17> has weaknesses {'radiation', 'bludgeoning'} and immunities set()
group <Group immune 2 n0=989 ep=24725 init=3 nc=989> has weaknesses {'slashing', 'bludgeoning'} and immunities {'fire'}
group <Group infection 1 n0=801 ep=92916 init=1 nc=801> has weaknesses {'radiation'} and immunities set()
group <Group infection 2 n0=4485 ep=53820 init=4 nc=4485> has weaknesses {'cold', 'fire'} and immunities {'radiation'}


In [1068]:
b.choose_targets()

2019-01-11 23:59:42,088 DEBUG    [day24.choose_targets:52] target choosing order is [<Group immune 1 n0=17 ep=76619 init=2 nc=17>, <Group immune 2 n0=989 ep=24725 init=3 nc=989>]
2019-01-11 23:59:42,098 DEBUG    [day24.choose_target:60] attempting to choose target for <Group immune 1 n0=17 ep=76619 init=2 nc=17> out of {<Group infection 2 n0=4485 ep=53820 init=4 nc=4485>, <Group infection 1 n0=801 ep=92916 init=1 nc=801>}
2019-01-11 23:59:42,105 DEBUG    [day24.possible_damage:87] group <Group immune 1 n0=17 ep=76619 init=2 nc=17> would do 153238 damage to <Group infection 2 n0=4485 ep=53820 init=4 nc=4485>
2019-01-11 23:59:42,110 DEBUG    [day24.possible_damage:87] group <Group immune 1 n0=17 ep=76619 init=2 nc=17> would do 76619 damage to <Group infection 1 n0=801 ep=92916 init=1 nc=801>
2019-01-11 23:59:42,119 DEBUG    [day24.possible_damage:87] group <Group immune 1 n0=17 ep=76619 init=2 nc=17> would do 153238 damage to <Group infection 2 n0=4485 ep=53820 init=4 nc=4485>
2019-01-11

In [1069]:
assert b.infection_army.groups[0].target == b.immune_army.groups[0]
assert b.infection_army.groups[1].target == b.immune_army.groups[1]
assert b.immune_army.groups[0].target == b.infection_army.groups[1]
assert b.immune_army.groups[1].target == b.infection_army.groups[0]

In [1070]:
b.run()

2019-01-11 23:59:43,422 DEBUG    [day24.choose_targets:52] target choosing order is [<Group immune 1 n0=17 ep=76619 init=2 nc=17>, <Group immune 2 n0=989 ep=24725 init=3 nc=989>]
2019-01-11 23:59:43,431 DEBUG    [day24.choose_target:60] attempting to choose target for <Group immune 1 n0=17 ep=76619 init=2 nc=17> out of {<Group infection 2 n0=4485 ep=53820 init=4 nc=4485>, <Group infection 1 n0=801 ep=92916 init=1 nc=801>}
2019-01-11 23:59:43,438 DEBUG    [day24.possible_damage:87] group <Group immune 1 n0=17 ep=76619 init=2 nc=17> would do 153238 damage to <Group infection 2 n0=4485 ep=53820 init=4 nc=4485>
2019-01-11 23:59:43,443 DEBUG    [day24.possible_damage:87] group <Group immune 1 n0=17 ep=76619 init=2 nc=17> would do 76619 damage to <Group infection 1 n0=801 ep=92916 init=1 nc=801>
2019-01-11 23:59:43,449 DEBUG    [day24.possible_damage:87] group <Group immune 1 n0=17 ep=76619 init=2 nc=17> would do 153238 damage to <Group infection 2 n0=4485 ep=53820 init=4 nc=4485>
2019-01-11

2019-01-11 23:59:43,583 DEBUG    [day24.take_damage:113] 82160 damage remaining
2019-01-11 23:59:43,584 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:43,585 DEBUG    [day24.take_damage:113] 80886 damage remaining
2019-01-11 23:59:43,586 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:43,587 DEBUG    [day24.take_damage:113] 79612 damage remaining
2019-01-11 23:59:43,588 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:43,590 DEBUG    [day24.take_damage:113] 78338 damage remaining
2019-01-11 23:59:43,591 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:43,592 DEBUG    [day24.take_damage:113] 77064 damage remaining
2019-01-11 23:59:43,594 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:43,595 DEBUG    [day24.take_

2019-01-11 23:59:43,725 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:43,727 DEBUG    [day24.take_damage:113] 27378 damage remaining
2019-01-11 23:59:43,728 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:43,729 DEBUG    [day24.take_damage:113] 26104 damage remaining
2019-01-11 23:59:43,730 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:43,732 DEBUG    [day24.take_damage:113] 24830 damage remaining
2019-01-11 23:59:43,733 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:43,734 DEBUG    [day24.take_damage:113] 23556 damage remaining
2019-01-11 23:59:43,736 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:43,737 DEBUG    [day24.take_damage:113] 22282 damage remaining
2019-01-11 23:59:43,738 DEBUG    [day24.take_

2019-01-11 23:59:43,855 DEBUG    [day24.take_damage:113] 120667 damage remaining
2019-01-11 23:59:43,856 DEBUG    [day24.take_damage:111] killing unit <Unit hp=2961 attack=12 slashing init=4>
2019-01-11 23:59:43,857 DEBUG    [day24.take_damage:113] 117706 damage remaining
2019-01-11 23:59:43,858 DEBUG    [day24.take_damage:111] killing unit <Unit hp=2961 attack=12 slashing init=4>
2019-01-11 23:59:43,859 DEBUG    [day24.take_damage:113] 114745 damage remaining
2019-01-11 23:59:43,861 DEBUG    [day24.take_damage:111] killing unit <Unit hp=2961 attack=12 slashing init=4>
2019-01-11 23:59:43,862 DEBUG    [day24.take_damage:113] 111784 damage remaining
2019-01-11 23:59:43,864 DEBUG    [day24.take_damage:111] killing unit <Unit hp=2961 attack=12 slashing init=4>
2019-01-11 23:59:43,865 DEBUG    [day24.take_damage:113] 108823 damage remaining
2019-01-11 23:59:43,866 DEBUG    [day24.take_damage:111] killing unit <Unit hp=2961 attack=12 slashing init=4>
2019-01-11 23:59:43,867 DEBUG    [day24.

2019-01-11 23:59:43,984 DEBUG    [day24.attack:94] attacking target <Group immune 1 n0=17 ep=76619 init=2 nc=17>
2019-01-11 23:59:43,986 DEBUG    [day24.take_damage:105] 184904 damage remaining
2019-01-11 23:59:43,988 DEBUG    [day24.take_damage:111] killing unit <Unit hp=5390 attack=4507 fire init=2>
2019-01-11 23:59:43,989 DEBUG    [day24.take_damage:113] 179514 damage remaining
2019-01-11 23:59:43,990 DEBUG    [day24.take_damage:111] killing unit <Unit hp=5390 attack=4507 fire init=2>
2019-01-11 23:59:43,991 DEBUG    [day24.take_damage:113] 174124 damage remaining
2019-01-11 23:59:43,993 DEBUG    [day24.take_damage:111] killing unit <Unit hp=5390 attack=4507 fire init=2>
2019-01-11 23:59:43,995 DEBUG    [day24.take_damage:113] 168734 damage remaining
2019-01-11 23:59:43,997 DEBUG    [day24.take_damage:111] killing unit <Unit hp=5390 attack=4507 fire init=2>
2019-01-11 23:59:43,998 DEBUG    [day24.take_damage:113] 163344 damage remaining
2019-01-11 23:59:43,999 DEBUG    [day24.take_d

...

2019-01-11 23:59:46,319 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:46,320 DEBUG    [day24.take_damage:113] 172506 damage remaining
2019-01-11 23:59:46,321 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:46,323 DEBUG    [day24.take_damage:113] 171232 damage remaining
2019-01-11 23:59:46,324 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:46,325 DEBUG    [day24.take_damage:113] 169958 damage remaining
2019-01-11 23:59:46,327 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:46,328 DEBUG    [day24.take_damage:113] 168684 damage remaining
2019-01-11 23:59:46,330 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:46,331 DEBUG    [day24.take_damage:113] 167410 damage remaining
2019-01-11 23:59:46,332 DEBUG    [day24.

2019-01-11 23:59:46,421 DEBUG    [day24.take_damage:113] 118998 damage remaining
2019-01-11 23:59:46,422 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:46,423 DEBUG    [day24.take_damage:113] 117724 damage remaining
2019-01-11 23:59:46,424 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:46,425 DEBUG    [day24.take_damage:113] 116450 damage remaining
2019-01-11 23:59:46,426 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:46,427 DEBUG    [day24.take_damage:113] 115176 damage remaining
2019-01-11 23:59:46,428 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:46,429 DEBUG    [day24.take_damage:113] 113902 damage remaining
2019-01-11 23:59:46,431 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:46,432 DEBUG    [day24.

2019-01-11 23:59:46,512 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:46,513 DEBUG    [day24.take_damage:113] 64216 damage remaining
2019-01-11 23:59:46,513 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:46,514 DEBUG    [day24.take_damage:113] 62942 damage remaining
2019-01-11 23:59:46,515 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:46,516 DEBUG    [day24.take_damage:113] 61668 damage remaining
2019-01-11 23:59:46,517 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:46,518 DEBUG    [day24.take_damage:113] 60394 damage remaining
2019-01-11 23:59:46,518 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:46,519 DEBUG    [day24.take_damage:113] 59120 damage remaining
2019-01-11 23:59:46,520 DEBUG    [day24.take_

2019-01-11 23:59:46,618 DEBUG    [day24.take_damage:113] 10708 damage remaining
2019-01-11 23:59:46,619 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:46,621 DEBUG    [day24.take_damage:113] 9434 damage remaining
2019-01-11 23:59:46,622 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:46,623 DEBUG    [day24.take_damage:113] 8160 damage remaining
2019-01-11 23:59:46,624 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:46,625 DEBUG    [day24.take_damage:113] 6886 damage remaining
2019-01-11 23:59:46,627 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:46,628 DEBUG    [day24.take_damage:113] 5612 damage remaining
2019-01-11 23:59:46,629 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:46,631 DEBUG    [day24.take_dama

2019-01-11 23:59:46,742 DEBUG    [day24.take_damage:113] 162314 damage remaining
2019-01-11 23:59:46,743 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:46,744 DEBUG    [day24.take_damage:113] 161040 damage remaining
2019-01-11 23:59:46,745 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:46,746 DEBUG    [day24.take_damage:113] 159766 damage remaining
2019-01-11 23:59:46,747 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:46,749 DEBUG    [day24.take_damage:113] 158492 damage remaining
2019-01-11 23:59:46,750 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:46,751 DEBUG    [day24.take_damage:113] 157218 damage remaining
2019-01-11 23:59:46,752 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:46,753 DEBUG    [day24.

In [1071]:
b.winner

<Army type=infection n=5286 n_living=5216>

In [1072]:
b.infection_army.num_living_units

5216

In [1073]:
b.winning_score

5216

In [1074]:
def test_q_1():
    LOGGER.setLevel(logging.DEBUG)
    assert q_1(test_data) == 5216
    LOGGER.setLevel(logging.INFO)

In [1075]:
test_q_1()

2019-01-11 23:59:49,627 DEBUG    [day24.init_from_summary:16] initalizing army from summary block
2019-01-11 23:59:49,630 DEBUG    [day24.init_from_summary:29] parsing summary line: 17 units each with 5390 hit points (weak to radiation, bludgeoning) with an attack that does 4507 fire damage at initiative 2
2019-01-11 23:59:49,633 DEBUG    [day24.init_from_summary:41] record has no immunities
2019-01-11 23:59:49,635 DEBUG    [day24.init_from_summary:29] parsing summary line: 989 units each with 1274 hit points (immune to fire; weak to bludgeoning, slashing) with an attack that does 25 slashing damage at initiative 3
2019-01-11 23:59:49,639 DEBUG    [day24.init_from_summary:16] initalizing army from summary block
2019-01-11 23:59:49,642 DEBUG    [day24.init_from_summary:29] parsing summary line: 801 units each with 4706 hit points (weak to radiation) with an attack that does 116 bludgeoning damage at initiative 1
2019-01-11 23:59:49,646 DEBUG    [day24.init_from_summary:41] record has no

2019-01-11 23:59:49,866 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:49,868 DEBUG    [day24.take_damage:113] 89804 damage remaining
2019-01-11 23:59:49,870 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:49,872 DEBUG    [day24.take_damage:113] 88530 damage remaining
2019-01-11 23:59:49,874 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:49,876 DEBUG    [day24.take_damage:113] 87256 damage remaining
2019-01-11 23:59:49,877 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:49,879 DEBUG    [day24.take_damage:113] 85982 damage remaining
2019-01-11 23:59:49,881 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:49,883 DEBUG    [day24.take_damage:113] 84708 damage remaining
2019-01-11 23:59:49,885 DEBUG    [day24.take_

2019-01-11 23:59:49,986 DEBUG    [day24.take_damage:113] 36296 damage remaining
2019-01-11 23:59:49,987 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:49,988 DEBUG    [day24.take_damage:113] 35022 damage remaining
2019-01-11 23:59:49,989 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:49,990 DEBUG    [day24.take_damage:113] 33748 damage remaining
2019-01-11 23:59:49,991 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:49,993 DEBUG    [day24.take_damage:113] 32474 damage remaining
2019-01-11 23:59:49,994 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:49,995 DEBUG    [day24.take_damage:113] 31200 damage remaining
2019-01-11 23:59:49,997 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:49,998 DEBUG    [day24.take_

2019-01-11 23:59:50,106 DEBUG    [day24.take_damage:113] 138433 damage remaining
2019-01-11 23:59:50,107 DEBUG    [day24.take_damage:111] killing unit <Unit hp=2961 attack=12 slashing init=4>
2019-01-11 23:59:50,108 DEBUG    [day24.take_damage:113] 135472 damage remaining
2019-01-11 23:59:50,109 DEBUG    [day24.take_damage:111] killing unit <Unit hp=2961 attack=12 slashing init=4>
2019-01-11 23:59:50,110 DEBUG    [day24.take_damage:113] 132511 damage remaining
2019-01-11 23:59:50,111 DEBUG    [day24.take_damage:111] killing unit <Unit hp=2961 attack=12 slashing init=4>
2019-01-11 23:59:50,113 DEBUG    [day24.take_damage:113] 129550 damage remaining
2019-01-11 23:59:50,114 DEBUG    [day24.take_damage:111] killing unit <Unit hp=2961 attack=12 slashing init=4>
2019-01-11 23:59:50,115 DEBUG    [day24.take_damage:113] 126589 damage remaining
2019-01-11 23:59:50,116 DEBUG    [day24.take_damage:111] killing unit <Unit hp=2961 attack=12 slashing init=4>
2019-01-11 23:59:50,117 DEBUG    [day24.

2019-01-11 23:59:50,228 DEBUG    [day24.take_damage:111] killing unit <Unit hp=2961 attack=12 slashing init=4>
2019-01-11 23:59:50,230 DEBUG    [day24.take_damage:113] 11110 damage remaining
2019-01-11 23:59:50,232 DEBUG    [day24.take_damage:111] killing unit <Unit hp=2961 attack=12 slashing init=4>
2019-01-11 23:59:50,234 DEBUG    [day24.take_damage:113] 8149 damage remaining
2019-01-11 23:59:50,235 DEBUG    [day24.take_damage:111] killing unit <Unit hp=2961 attack=12 slashing init=4>
2019-01-11 23:59:50,236 DEBUG    [day24.take_damage:113] 5188 damage remaining
2019-01-11 23:59:50,237 DEBUG    [day24.take_damage:111] killing unit <Unit hp=2961 attack=12 slashing init=4>
2019-01-11 23:59:50,239 DEBUG    [day24.take_damage:113] 2227 damage remaining
2019-01-11 23:59:50,240 DEBUG    [day24.take_damage:117] insufficient damage to kill unit, group damage is done
2019-01-11 23:59:50,243 DEBUG    [day24.take_damage:119] group <Group infection 2 n0=4485 ep=53208 init=4 nc=4434> just lost 51

...


2019-01-11 23:59:52,635 DEBUG    [day24.take_damage:113] 172506 damage remaining
2019-01-11 23:59:52,636 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:52,638 DEBUG    [day24.take_damage:113] 171232 damage remaining
2019-01-11 23:59:52,639 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:52,641 DEBUG    [day24.take_damage:113] 169958 damage remaining
2019-01-11 23:59:52,643 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:52,644 DEBUG    [day24.take_damage:113] 168684 damage remaining
2019-01-11 23:59:52,645 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:52,646 DEBUG    [day24.take_damage:113] 167410 damage remaining
2019-01-11 23:59:52,647 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:52,648 DEBUG    [day24

2019-01-11 23:59:52,739 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:52,740 DEBUG    [day24.take_damage:113] 117724 damage remaining
2019-01-11 23:59:52,741 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:52,742 DEBUG    [day24.take_damage:113] 116450 damage remaining
2019-01-11 23:59:52,743 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:52,744 DEBUG    [day24.take_damage:113] 115176 damage remaining
2019-01-11 23:59:52,745 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:52,746 DEBUG    [day24.take_damage:113] 113902 damage remaining
2019-01-11 23:59:52,748 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:52,749 DEBUG    [day24.take_damage:113] 112628 damage remaining
2019-01-11 23:59:52,750 DEBUG    [day24.

2019-01-11 23:59:52,833 DEBUG    [day24.take_damage:113] 64216 damage remaining
2019-01-11 23:59:52,834 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:52,835 DEBUG    [day24.take_damage:113] 62942 damage remaining
2019-01-11 23:59:52,836 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:52,838 DEBUG    [day24.take_damage:113] 61668 damage remaining
2019-01-11 23:59:52,839 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:52,841 DEBUG    [day24.take_damage:113] 60394 damage remaining
2019-01-11 23:59:52,842 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:52,843 DEBUG    [day24.take_damage:113] 59120 damage remaining
2019-01-11 23:59:52,845 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:52,846 DEBUG    [day24.take_

2019-01-11 23:59:52,934 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:52,935 DEBUG    [day24.take_damage:113] 9434 damage remaining
2019-01-11 23:59:52,936 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:52,937 DEBUG    [day24.take_damage:113] 8160 damage remaining
2019-01-11 23:59:52,938 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:52,939 DEBUG    [day24.take_damage:113] 6886 damage remaining
2019-01-11 23:59:52,940 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:52,941 DEBUG    [day24.take_damage:113] 5612 damage remaining
2019-01-11 23:59:52,943 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:52,945 DEBUG    [day24.take_damage:113] 4338 damage remaining
2019-01-11 23:59:52,946 DEBUG    [day24.take_damag

2019-01-11 23:59:53,041 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:53,042 DEBUG    [day24.take_damage:113] 161040 damage remaining
2019-01-11 23:59:53,043 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:53,044 DEBUG    [day24.take_damage:113] 159766 damage remaining
2019-01-11 23:59:53,045 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:53,046 DEBUG    [day24.take_damage:113] 158492 damage remaining
2019-01-11 23:59:53,047 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:53,048 DEBUG    [day24.take_damage:113] 157218 damage remaining
2019-01-11 23:59:53,049 DEBUG    [day24.take_damage:111] killing unit <Unit hp=1274 attack=25 slashing init=3>
2019-01-11 23:59:53,050 DEBUG    [day24.take_damage:113] 155944 damage remaining
2019-01-11 23:59:53,051 DEBUG    [day24.

#### answer

In [1019]:
q_1(load_data())

2019-01-11 23:45:31,071 INFO     [day24.attack:45] infection deaths: 51
2019-01-11 23:45:31,073 INFO     [day24.attack:46] immune deaths: 121
2019-01-11 23:45:31,089 INFO     [day24.log:59] immune: 33,887, infection: 21,118
2019-01-11 23:45:31,929 INFO     [day24.attack:45] infection deaths: 50
2019-01-11 23:45:31,931 INFO     [day24.attack:46] immune deaths: 120
2019-01-11 23:45:31,964 INFO     [day24.log:59] immune: 33,767, infection: 21,068
2019-01-11 23:45:32,796 INFO     [day24.attack:45] infection deaths: 49
2019-01-11 23:45:32,798 INFO     [day24.attack:46] immune deaths: 120
2019-01-11 23:45:32,812 INFO     [day24.log:59] immune: 33,647, infection: 21,019
2019-01-11 23:45:33,651 INFO     [day24.attack:45] infection deaths: 49
2019-01-11 23:45:33,654 INFO     [day24.attack:46] immune deaths: 122
2019-01-11 23:45:33,693 INFO     [day24.log:59] immune: 33,525, infection: 20,970
2019-01-11 23:45:34,536 INFO     [day24.attack:45] infection deaths: 48
2019-01-11 23:45:34,538 INFO    

2019-01-11 23:46:02,225 INFO     [day24.attack:46] immune deaths: 105
2019-01-11 23:46:02,240 INFO     [day24.log:59] immune: 29,748, infection: 19,529
2019-01-11 23:46:03,025 INFO     [day24.attack:45] infection deaths: 42
2019-01-11 23:46:03,027 INFO     [day24.attack:46] immune deaths: 103
2019-01-11 23:46:03,042 INFO     [day24.log:59] immune: 29,645, infection: 19,487
2019-01-11 23:46:03,763 INFO     [day24.attack:45] infection deaths: 41
2019-01-11 23:46:03,765 INFO     [day24.attack:46] immune deaths: 103
2019-01-11 23:46:03,815 INFO     [day24.log:59] immune: 29,542, infection: 19,446
2019-01-11 23:46:04,603 INFO     [day24.attack:45] infection deaths: 41
2019-01-11 23:46:04,604 INFO     [day24.attack:46] immune deaths: 103
2019-01-11 23:46:04,621 INFO     [day24.log:59] immune: 29,439, infection: 19,405
2019-01-11 23:46:05,324 INFO     [day24.attack:45] infection deaths: 41
2019-01-11 23:46:05,326 INFO     [day24.attack:46] immune deaths: 103
2019-01-11 23:46:05,344 INFO     [

2019-01-11 23:46:29,102 INFO     [day24.log:59] immune: 26,187, infection: 18,100
2019-01-11 23:46:29,780 INFO     [day24.attack:45] infection deaths: 38
2019-01-11 23:46:29,781 INFO     [day24.attack:46] immune deaths: 87
2019-01-11 23:46:29,834 INFO     [day24.log:59] immune: 26,100, infection: 18,062
2019-01-11 23:46:30,592 INFO     [day24.attack:45] infection deaths: 38
2019-01-11 23:46:30,594 INFO     [day24.attack:46] immune deaths: 87
2019-01-11 23:46:30,611 INFO     [day24.log:59] immune: 26,013, infection: 18,024
2019-01-11 23:46:31,303 INFO     [day24.attack:45] infection deaths: 38
2019-01-11 23:46:31,305 INFO     [day24.attack:46] immune deaths: 87
2019-01-11 23:46:31,345 INFO     [day24.log:59] immune: 25,926, infection: 17,986
2019-01-11 23:46:32,020 INFO     [day24.attack:45] infection deaths: 38
2019-01-11 23:46:32,022 INFO     [day24.attack:46] immune deaths: 87
2019-01-11 23:46:32,039 INFO     [day24.log:59] immune: 25,839, infection: 17,948
2019-01-11 23:46:32,718 IN

2019-01-11 23:46:55,939 INFO     [day24.attack:46] immune deaths: 84
2019-01-11 23:46:55,956 INFO     [day24.log:59] immune: 23,029, infection: 16,766
2019-01-11 23:46:56,667 INFO     [day24.attack:45] infection deaths: 33
2019-01-11 23:46:56,668 INFO     [day24.attack:46] immune deaths: 84
2019-01-11 23:46:56,685 INFO     [day24.log:59] immune: 22,945, infection: 16,733
2019-01-11 23:46:57,373 INFO     [day24.attack:45] infection deaths: 32
2019-01-11 23:46:57,375 INFO     [day24.attack:46] immune deaths: 84
2019-01-11 23:46:57,392 INFO     [day24.log:59] immune: 22,861, infection: 16,701
2019-01-11 23:46:58,042 INFO     [day24.attack:45] infection deaths: 32
2019-01-11 23:46:58,043 INFO     [day24.attack:46] immune deaths: 84
2019-01-11 23:46:58,059 INFO     [day24.log:59] immune: 22,777, infection: 16,669
2019-01-11 23:46:58,756 INFO     [day24.attack:45] infection deaths: 32
2019-01-11 23:46:58,758 INFO     [day24.attack:46] immune deaths: 84
2019-01-11 23:46:58,795 INFO     [day24

2019-01-11 23:47:21,916 INFO     [day24.attack:45] infection deaths: 20
2019-01-11 23:47:21,919 INFO     [day24.attack:46] immune deaths: 84
2019-01-11 23:47:21,938 INFO     [day24.log:59] immune: 19,921, infection: 15,695
2019-01-11 23:47:22,581 INFO     [day24.attack:45] infection deaths: 20
2019-01-11 23:47:22,583 INFO     [day24.attack:46] immune deaths: 84
2019-01-11 23:47:22,644 INFO     [day24.log:59] immune: 19,837, infection: 15,675
2019-01-11 23:47:23,343 INFO     [day24.attack:45] infection deaths: 20
2019-01-11 23:47:23,345 INFO     [day24.attack:46] immune deaths: 84
2019-01-11 23:47:23,362 INFO     [day24.log:59] immune: 19,753, infection: 15,655
2019-01-11 23:47:24,002 INFO     [day24.attack:45] infection deaths: 20
2019-01-11 23:47:24,004 INFO     [day24.attack:46] immune deaths: 84
2019-01-11 23:47:24,042 INFO     [day24.log:59] immune: 19,669, infection: 15,635
2019-01-11 23:47:24,672 INFO     [day24.attack:45] infection deaths: 20
2019-01-11 23:47:24,674 INFO     [da

...

2019-01-11 23:49:46,253 INFO     [day24.log:59] immune: 3,418, infection: 14,799
2019-01-11 23:49:46,784 INFO     [day24.attack:45] infection deaths: 0
2019-01-11 23:49:46,786 INFO     [day24.attack:46] immune deaths: 48
2019-01-11 23:49:46,801 INFO     [day24.log:59] immune: 3,370, infection: 14,799
2019-01-11 23:49:47,217 INFO     [day24.attack:45] infection deaths: 0
2019-01-11 23:49:47,219 INFO     [day24.attack:46] immune deaths: 45
2019-01-11 23:49:47,236 INFO     [day24.log:59] immune: 3,325, infection: 14,799
2019-01-11 23:49:47,698 INFO     [day24.attack:45] infection deaths: 0
2019-01-11 23:49:47,700 INFO     [day24.attack:46] immune deaths: 44
2019-01-11 23:49:47,718 INFO     [day24.log:59] immune: 3,281, infection: 14,799
2019-01-11 23:49:48,107 INFO     [day24.attack:45] infection deaths: 0
2019-01-11 23:49:48,109 INFO     [day24.attack:46] immune deaths: 45
2019-01-11 23:49:48,127 INFO     [day24.log:59] immune: 3,236, infection: 14,799
2019-01-11 23:49:48,546 INFO     [d

2019-01-11 23:50:02,379 INFO     [day24.log:59] immune: 1,726, infection: 14,799
2019-01-11 23:50:02,765 INFO     [day24.attack:45] infection deaths: 0
2019-01-11 23:50:02,767 INFO     [day24.attack:46] immune deaths: 50
2019-01-11 23:50:02,786 INFO     [day24.log:59] immune: 1,676, infection: 14,799
2019-01-11 23:50:03,201 INFO     [day24.attack:45] infection deaths: 0
2019-01-11 23:50:03,203 INFO     [day24.attack:46] immune deaths: 50
2019-01-11 23:50:03,220 INFO     [day24.log:59] immune: 1,626, infection: 14,799
2019-01-11 23:50:03,607 INFO     [day24.attack:45] infection deaths: 0
2019-01-11 23:50:03,608 INFO     [day24.attack:46] immune deaths: 50
2019-01-11 23:50:03,623 INFO     [day24.log:59] immune: 1,576, infection: 14,799
2019-01-11 23:50:04,031 INFO     [day24.attack:45] infection deaths: 0
2019-01-11 23:50:04,033 INFO     [day24.attack:46] immune deaths: 50
2019-01-11 23:50:04,048 INFO     [day24.log:59] immune: 1,526, infection: 14,799
2019-01-11 23:50:04,428 INFO     [d

2019-01-11 23:50:13,998 INFO     [day24.log:59] immune: 509, infection: 14,799
2019-01-11 23:50:14,144 INFO     [day24.attack:45] infection deaths: 0
2019-01-11 23:50:14,146 INFO     [day24.attack:46] immune deaths: 10
2019-01-11 23:50:14,160 INFO     [day24.log:59] immune: 499, infection: 14,799
2019-01-11 23:50:14,313 INFO     [day24.attack:45] infection deaths: 0
2019-01-11 23:50:14,315 INFO     [day24.attack:46] immune deaths: 10
2019-01-11 23:50:14,327 INFO     [day24.log:59] immune: 489, infection: 14,799
2019-01-11 23:50:14,476 INFO     [day24.attack:45] infection deaths: 0
2019-01-11 23:50:14,478 INFO     [day24.attack:46] immune deaths: 10
2019-01-11 23:50:14,490 INFO     [day24.log:59] immune: 479, infection: 14,799
2019-01-11 23:50:14,644 INFO     [day24.attack:45] infection deaths: 0
2019-01-11 23:50:14,645 INFO     [day24.attack:46] immune deaths: 10
2019-01-11 23:50:14,660 INFO     [day24.log:59] immune: 469, infection: 14,799
2019-01-11 23:50:14,808 INFO     [day24.attac

2019-01-11 23:50:20,858 INFO     [day24.attack:45] infection deaths: 0
2019-01-11 23:50:20,861 INFO     [day24.attack:46] immune deaths: 10
2019-01-11 23:50:20,877 INFO     [day24.log:59] immune: 129, infection: 14,799
2019-01-11 23:50:21,028 INFO     [day24.attack:45] infection deaths: 0
2019-01-11 23:50:21,029 INFO     [day24.attack:46] immune deaths: 10
2019-01-11 23:50:21,042 INFO     [day24.log:59] immune: 119, infection: 14,799
2019-01-11 23:50:21,189 INFO     [day24.attack:45] infection deaths: 0
2019-01-11 23:50:21,191 INFO     [day24.attack:46] immune deaths: 10
2019-01-11 23:50:21,214 INFO     [day24.log:59] immune: 109, infection: 14,799
2019-01-11 23:50:21,363 INFO     [day24.attack:45] infection deaths: 0
2019-01-11 23:50:21,365 INFO     [day24.attack:46] immune deaths: 10
2019-01-11 23:50:21,378 INFO     [day24.log:59] immune: 99, infection: 14,799
2019-01-11 23:50:21,563 INFO     [day24.attack:45] infection deaths: 0
2019-01-11 23:50:21,565 INFO     [day24.attack:46] imm

14799

## part 2

### problem statement:

> Things aren't looking good for the reindeer. The man asks whether more milk and cookies would help you think.
> 
> If only you could give the reindeer's immune system a boost, you might be able to change the outcome of the combat.
> 
> A boost is an integer increase in immune system units' attack damage. For example, if you were to boost the above example's immune system's units by 1570, the armies would instead look like this:
> 
>     Immune System:
>     17 units each with 5390 hit points (weak to radiation, bludgeoning) with
>      an attack that does 6077 fire damage at initiative 2
>     989 units each with 1274 hit points (immune to fire; weak to bludgeoning,
>      slashing) with an attack that does 1595 slashing damage at initiative 3
>     
>     Infection:
>     801 units each with 4706 hit points (weak to radiation) with an attack
>      that does 116 bludgeoning damage at initiative 1
>     4485 units each with 2961 hit points (immune to radiation; weak to fire,
>      cold) with an attack that does 12 slashing damage at initiative 4
>
> With this boost, the combat proceeds differently:
> 
>     Immune System:
>     Group 2 contains 989 units
>     Group 1 contains 17 units
>     Infection:
>     Group 1 contains 801 units
>     Group 2 contains 4485 units
>     
>     Infection group 1 would deal defending group 2 185832 damage
>     Infection group 1 would deal defending group 1 185832 damage
>     Infection group 2 would deal defending group 1 53820 damage
>     Immune System group 2 would deal defending group 1 1577455 damage
>     Immune System group 2 would deal defending group 2 1577455 damage
>     Immune System group 1 would deal defending group 2 206618 damage
>     
>     Infection group 2 attacks defending group 1, killing 9 units
>     Immune System group 2 attacks defending group 1, killing 335 units
>     Immune System group 1 attacks defending group 2, killing 32 units
>     Infection group 1 attacks defending group 2, killing 84 units
>
>     Immune System:
>     Group 2 contains 905 units
>     Group 1 contains 8 units
>     Infection:
>     Group 1 contains 466 units
>     Group 2 contains 4453 units
>     
>     Infection group 1 would deal defending group 2 108112 damage
>     Infection group 1 would deal defending group 1 108112 damage
>     Infection group 2 would deal defending group 1 53436 damage
>     Immune System group 2 would deal defending group 1 1443475 damage
>     Immune System group 2 would deal defending group 2 1443475 damage
>     Immune System group 1 would deal defending group 2 97232 damage
>     
>     Infection group 2 attacks defending group 1, killing 8 units
>     Immune System group 2 attacks defending group 1, killing 306 units
>     Infection group 1 attacks defending group 2, killing 29 units
>
>     Immune System:
>     Group 2 contains 876 units
>     Infection:
>     Group 2 contains 4453 units
>     Group 1 contains 160 units
>     
>     Infection group 2 would deal defending group 2 106872 damage
>     Immune System group 2 would deal defending group 2 1397220 damage
>     Immune System group 2 would deal defending group 1 1397220 damage
>     
>     Infection group 2 attacks defending group 2, killing 83 units
>     Immune System group 2 attacks defending group 2, killing 427 units
> 
> After a few fights...
> 
>     Immune System:
>     Group 2 contains 64 units
>     Infection:
>     Group 2 contains 214 units
>     Group 1 contains 19 units
>     
>     Infection group 2 would deal defending group 2 5136 damage
>     Immune System group 2 would deal defending group 2 102080 damage
>     Immune System group 2 would deal defending group 1 102080 damage
>     
>     Infection group 2 attacks defending group 2, killing 4 units
>     Immune System group 2 attacks defending group 2, killing 32 units
> 
>     Immune System:
>     Group 2 contains 60 units
>     Infection:
>     Group 1 contains 19 units
>     Group 2 contains 182 units
>     
>     Infection group 1 would deal defending group 2 4408 damage
>     Immune System group 2 would deal defending group 1 95700 damage
>     Immune System group 2 would deal defending group 2 95700 damage
>     
>     Immune System group 2 attacks defending group 1, killing 19 units
> 
>     Immune System:
>     Group 2 contains 60 units
>     Infection:
>     Group 2 contains 182 units
>     
>     Infection group 2 would deal defending group 2 4368 damage
>     Immune System group 2 would deal defending group 2 95700 damage
>     
>     Infection group 2 attacks defending group 2, killing 3 units
>     Immune System group 2 attacks defending group 2, killing 30 units
>
> After a few more fights...
> 
>     Immune System:
>     Group 2 contains 51 units
>     Infection:
>     Group 2 contains 40 units
>     
>     Infection group 2 would deal defending group 2 960 damage
>     Immune System group 2 would deal defending group 2 81345 damage
>     
>     Infection group 2 attacks defending group 2, killing 0 units
>     Immune System group 2 attacks defending group 2, killing 27 units
> 
>     Immune System:
>     Group 2 contains 51 units
>     Infection:
>     Group 2 contains 13 units
>     
>     Infection group 2 would deal defending group 2 312 damage
>     Immune System group 2 would deal defending group 2 81345 damage
>     
>     Infection group 2 attacks defending group 2, killing 0 units
>     Immune System group 2 attacks defending group 2, killing 13 units
>
>     Immune System:
>     Group 2 contains 51 units
>     Infection:
>     No groups remain.
>
> This boost would allow the immune system's armies to win! It would be left with 51 units.
> 
> You don't even know how you could boost the reindeer's immune system or what effect it might have, so you need to be cautious and find the smallest boost that would allow the immune system to win.
> 
> How many units does the immune system have left after getting the smallest boost it needs to win?

#### function def

In [1098]:
def q_2(data, boost_0=None, boost_1=None):
    if boost_0 is None and boost_1 is None:
        # burn-in from 2 ** 24
        boost = 2 ** 24
        LOGGER.warning('boost value: {}'.format(boost))
        b = Battle(data, immune_boost=boost)
        b.run()
        assert b.winner == b.immune_army

        while b.winner == b.immune_army:
            boost_prev = boost
            boost /= 2
            LOGGER.warning('boost value: {}'.format(boost))
            b = Battle(data, immune_boost=boost)
            b.run()
    
        # we've "burned down" to the firt boost for which
        # we didn't win. do a binary search between this value and
        # boost_prev to find the answer
        boost_0 = boost
        boost_1 = boost_prev
    elif boost_0 is None or boost_1 is None:
        raise ValueError('both boost params must be provided')
        
    boost_mp = (boost_1 + boost_0) // 2
    scores = {}
    while True:
        LOGGER.warning('boost boundaries: {} {} {}'.format(boost_0, boost_mp, boost_1))
        b = Battle(data, immune_boost=boost_mp)
        b.run()
        
        if b.winner == b.immune_army:
            scores[boost_mp] = b.winning_score
            # reset to lower half
            boost_1 = boost_mp
        else:
            # reset to upper half
            boost_0 = boost_mp
            
        boost_mp = (boost_1 + boost_0) // 2
        
        if boost_mp in [boost_0, boost_1]:
            break
    
    # the current value of boost_1 is the smallest successful boost value
    # return the winning score
    return scores[boost_1]

#### tests

In [1099]:
LOGGER.setLevel(logging.WARNING)
assert q_2(test_data) == 51

2019-01-12 01:31:33,608 WARNING  [day24.q_2:5] boost value: 16777216
2019-01-12 01:31:33,779 WARNING  [day24.q_2:13] boost value: 8388608.0
2019-01-12 01:31:33,874 WARNING  [day24.q_2:13] boost value: 4194304.0
2019-01-12 01:31:33,970 WARNING  [day24.q_2:13] boost value: 2097152.0
2019-01-12 01:31:34,073 WARNING  [day24.q_2:13] boost value: 1048576.0
2019-01-12 01:31:34,165 WARNING  [day24.q_2:13] boost value: 524288.0
2019-01-12 01:31:34,297 WARNING  [day24.q_2:13] boost value: 262144.0
2019-01-12 01:31:34,429 WARNING  [day24.q_2:13] boost value: 131072.0
2019-01-12 01:31:34,564 WARNING  [day24.q_2:13] boost value: 65536.0
2019-01-12 01:31:34,700 WARNING  [day24.q_2:13] boost value: 32768.0
2019-01-12 01:31:34,831 WARNING  [day24.q_2:13] boost value: 16384.0
2019-01-12 01:31:34,958 WARNING  [day24.q_2:13] boost value: 8192.0
2019-01-12 01:31:35,166 WARNING  [day24.q_2:13] boost value: 4096.0
2019-01-12 01:31:35,449 WARNING  [day24.q_2:13] boost value: 2048.0
2019-01-12 01:31:36,110 WA

#### answer

In [1101]:
# ran this into error, that's how I know 32 is a successful value
# 16 is a fail and 24 is a stalemate
LOGGER.setLevel(logging.INFO)
q_2(load_data(), boost_0=16, boost_1=32)

2019-01-12 01:32:17,277 WARNING  [day24.q_2:28] boost boundaries: 16 24 32
2019-01-12 01:32:18,477 INFO     [day24.attack:48] infection deaths: 142
2019-01-12 01:32:18,479 INFO     [day24.attack:49] immune deaths: 126
2019-01-12 01:32:18,500 INFO     [day24.log:62] immune: 33,882, infection: 21,027
2019-01-12 01:32:19,359 INFO     [day24.attack:48] infection deaths: 142
2019-01-12 01:32:19,361 INFO     [day24.attack:49] immune deaths: 123
2019-01-12 01:32:19,375 INFO     [day24.log:62] immune: 33,759, infection: 20,885
2019-01-12 01:32:20,231 INFO     [day24.attack:48] infection deaths: 141
2019-01-12 01:32:20,232 INFO     [day24.attack:49] immune deaths: 118
2019-01-12 01:32:20,252 INFO     [day24.log:62] immune: 33,641, infection: 20,744
2019-01-12 01:32:21,180 INFO     [day24.attack:48] infection deaths: 141
2019-01-12 01:32:21,182 INFO     [day24.attack:49] immune deaths: 113
2019-01-12 01:32:21,197 INFO     [day24.log:62] immune: 33,528, infection: 20,603
2019-01-12 01:32:22,111 I

2019-01-12 01:32:50,820 INFO     [day24.attack:48] infection deaths: 122
2019-01-12 01:32:50,821 INFO     [day24.attack:49] immune deaths: 85
2019-01-12 01:32:50,837 INFO     [day24.log:62] immune: 29,968, infection: 16,384
2019-01-12 01:32:51,702 INFO     [day24.attack:48] infection deaths: 122
2019-01-12 01:32:51,703 INFO     [day24.attack:49] immune deaths: 85
2019-01-12 01:32:51,720 INFO     [day24.log:62] immune: 29,883, infection: 16,262
2019-01-12 01:32:52,534 INFO     [day24.attack:48] infection deaths: 104
2019-01-12 01:32:52,536 INFO     [day24.attack:49] immune deaths: 85
2019-01-12 01:32:52,551 INFO     [day24.log:62] immune: 29,798, infection: 16,158
2019-01-12 01:32:53,273 INFO     [day24.attack:48] infection deaths: 90
2019-01-12 01:32:53,275 INFO     [day24.attack:49] immune deaths: 84
2019-01-12 01:32:53,292 INFO     [day24.log:62] immune: 29,714, infection: 16,068
2019-01-12 01:32:54,017 INFO     [day24.attack:48] infection deaths: 90
2019-01-12 01:32:54,018 INFO     

2019-01-12 01:33:18,415 INFO     [day24.log:62] immune: 27,246, infection: 13,956
2019-01-12 01:33:19,021 INFO     [day24.attack:48] infection deaths: 29
2019-01-12 01:33:19,023 INFO     [day24.attack:49] immune deaths: 68
2019-01-12 01:33:19,039 INFO     [day24.log:62] immune: 27,178, infection: 13,927
2019-01-12 01:33:19,638 INFO     [day24.attack:48] infection deaths: 29
2019-01-12 01:33:19,640 INFO     [day24.attack:49] immune deaths: 68
2019-01-12 01:33:19,657 INFO     [day24.log:62] immune: 27,110, infection: 13,898
2019-01-12 01:33:20,256 INFO     [day24.attack:48] infection deaths: 29
2019-01-12 01:33:20,257 INFO     [day24.attack:49] immune deaths: 67
2019-01-12 01:33:20,274 INFO     [day24.log:62] immune: 27,043, infection: 13,869
2019-01-12 01:33:20,877 INFO     [day24.attack:48] infection deaths: 29
2019-01-12 01:33:20,878 INFO     [day24.attack:49] immune deaths: 66
2019-01-12 01:33:20,895 INFO     [day24.log:62] immune: 26,977, infection: 13,840
2019-01-12 01:33:21,499 IN

2019-01-12 01:33:41,229 INFO     [day24.attack:49] immune deaths: 53
2019-01-12 01:33:41,261 INFO     [day24.log:62] immune: 24,921, infection: 13,005
2019-01-12 01:33:41,820 INFO     [day24.attack:48] infection deaths: 17
2019-01-12 01:33:41,821 INFO     [day24.attack:49] immune deaths: 59
2019-01-12 01:33:41,851 INFO     [day24.log:62] immune: 24,862, infection: 12,988
2019-01-12 01:33:42,377 INFO     [day24.attack:48] infection deaths: 17
2019-01-12 01:33:42,378 INFO     [day24.attack:49] immune deaths: 53
2019-01-12 01:33:42,408 INFO     [day24.log:62] immune: 24,809, infection: 12,971
2019-01-12 01:33:42,905 INFO     [day24.attack:48] infection deaths: 17
2019-01-12 01:33:42,907 INFO     [day24.attack:49] immune deaths: 58
2019-01-12 01:33:42,924 INFO     [day24.log:62] immune: 24,751, infection: 12,954
2019-01-12 01:33:43,489 INFO     [day24.attack:48] infection deaths: 17
2019-01-12 01:33:43,491 INFO     [day24.attack:49] immune deaths: 52
2019-01-12 01:33:43,511 INFO     [day24

2019-01-12 01:34:01,253 INFO     [day24.attack:48] infection deaths: 17
2019-01-12 01:34:01,255 INFO     [day24.attack:49] immune deaths: 49
2019-01-12 01:34:01,276 INFO     [day24.log:62] immune: 22,967, infection: 12,376
2019-01-12 01:34:01,767 INFO     [day24.attack:48] infection deaths: 16
2019-01-12 01:34:01,769 INFO     [day24.attack:49] immune deaths: 54
2019-01-12 01:34:01,785 INFO     [day24.log:62] immune: 22,913, infection: 12,360
2019-01-12 01:34:02,271 INFO     [day24.attack:48] infection deaths: 16
2019-01-12 01:34:02,273 INFO     [day24.attack:49] immune deaths: 49
2019-01-12 01:34:02,302 INFO     [day24.log:62] immune: 22,864, infection: 12,344
2019-01-12 01:34:02,820 INFO     [day24.attack:48] infection deaths: 16
2019-01-12 01:34:02,821 INFO     [day24.attack:49] immune deaths: 54
2019-01-12 01:34:02,838 INFO     [day24.log:62] immune: 22,810, infection: 12,328
2019-01-12 01:34:03,323 INFO     [day24.attack:48] infection deaths: 16
2019-01-12 01:34:03,324 INFO     [da

...

2019-01-12 02:36:22,522 INFO     [day24.attack:48] infection deaths: 1
2019-01-12 02:36:22,523 INFO     [day24.attack:49] immune deaths: 0
2019-01-12 02:36:22,538 INFO     [day24.log:62] immune: 4,428, infection: 113
2019-01-12 02:36:22,730 INFO     [day24.attack:48] infection deaths: 1
2019-01-12 02:36:22,731 INFO     [day24.attack:49] immune deaths: 0
2019-01-12 02:36:22,746 INFO     [day24.log:62] immune: 4,428, infection: 112
2019-01-12 02:36:22,936 INFO     [day24.attack:48] infection deaths: 1
2019-01-12 02:36:22,938 INFO     [day24.attack:49] immune deaths: 0
2019-01-12 02:36:22,952 INFO     [day24.log:62] immune: 4,428, infection: 111
2019-01-12 02:36:23,147 INFO     [day24.attack:48] infection deaths: 1
2019-01-12 02:36:23,149 INFO     [day24.attack:49] immune deaths: 0
2019-01-12 02:36:23,165 INFO     [day24.log:62] immune: 4,428, infection: 110
2019-01-12 02:36:23,358 INFO     [day24.attack:48] infection deaths: 1
2019-01-12 02:36:23,360 INFO     [day24.attack:49] immune dea

2019-01-12 02:36:30,939 INFO     [day24.log:62] immune: 4,428, infection: 76
2019-01-12 02:36:31,132 INFO     [day24.attack:48] infection deaths: 1
2019-01-12 02:36:31,133 INFO     [day24.attack:49] immune deaths: 0
2019-01-12 02:36:31,150 INFO     [day24.log:62] immune: 4,428, infection: 75
2019-01-12 02:36:31,341 INFO     [day24.attack:48] infection deaths: 1
2019-01-12 02:36:31,343 INFO     [day24.attack:49] immune deaths: 0
2019-01-12 02:36:31,360 INFO     [day24.log:62] immune: 4,428, infection: 74
2019-01-12 02:36:31,550 INFO     [day24.attack:48] infection deaths: 1
2019-01-12 02:36:31,551 INFO     [day24.attack:49] immune deaths: 0
2019-01-12 02:36:31,568 INFO     [day24.log:62] immune: 4,428, infection: 73
2019-01-12 02:36:31,758 INFO     [day24.attack:48] infection deaths: 1
2019-01-12 02:36:31,761 INFO     [day24.attack:49] immune deaths: 0
2019-01-12 02:36:31,792 INFO     [day24.log:62] immune: 4,428, infection: 72
2019-01-12 02:36:32,067 INFO     [day24.attack:48] infectio

2019-01-12 02:36:39,151 INFO     [day24.attack:49] immune deaths: 0
2019-01-12 02:36:39,166 INFO     [day24.log:62] immune: 4,428, infection: 38
2019-01-12 02:36:39,358 INFO     [day24.attack:48] infection deaths: 1
2019-01-12 02:36:39,359 INFO     [day24.attack:49] immune deaths: 0
2019-01-12 02:36:39,374 INFO     [day24.log:62] immune: 4,428, infection: 37
2019-01-12 02:36:39,563 INFO     [day24.attack:48] infection deaths: 1
2019-01-12 02:36:39,565 INFO     [day24.attack:49] immune deaths: 0
2019-01-12 02:36:39,585 INFO     [day24.log:62] immune: 4,428, infection: 36
2019-01-12 02:36:39,786 INFO     [day24.attack:48] infection deaths: 1
2019-01-12 02:36:39,788 INFO     [day24.attack:49] immune deaths: 0
2019-01-12 02:36:39,803 INFO     [day24.log:62] immune: 4,428, infection: 35
2019-01-12 02:36:40,000 INFO     [day24.attack:48] infection deaths: 1
2019-01-12 02:36:40,002 INFO     [day24.attack:49] immune deaths: 0
2019-01-12 02:36:40,017 INFO     [day24.log:62] immune: 4,428, infec

2019-01-12 02:36:47,808 INFO     [day24.attack:48] infection deaths: 1
2019-01-12 02:36:47,810 INFO     [day24.attack:49] immune deaths: 0
2019-01-12 02:36:47,824 INFO     [day24.log:62] immune: 4,428, infection: 0


4428

fin

In [1102]:
import re

def binary_search(f, lo=0, hi=None):
    """
    Returns a value x such that f(x) is true.
    Based on the values of f at lo and hi.
    Assert that f(lo) != f(hi).
    """
    lo_bool = f(lo)
    if hi is None:
        offset = 1
        while f(lo+offset) == lo_bool:
            offset *= 2
        hi = lo + offset
    else:
        assert f(hi) != lo_bool
    best_so_far = lo if lo_bool else hi
    while lo <= hi:
        mid = (hi + lo) // 2
        result = f(mid)
        if result:
            best_so_far = mid
        if result == lo_bool:
            lo = mid + 1
        else:
            hi = mid - 1
    return best_so_far


inp = """
Immune System:
4555 units each with 9688 hit points (immune to radiation; weak to bludgeoning) with an attack that does 17 radiation damage at initiative 1
2698 units each with 9598 hit points (immune to slashing) with an attack that does 29 slashing damage at initiative 16
4682 units each with 6161 hit points with an attack that does 13 radiation damage at initiative 19
8197 units each with 4985 hit points (weak to cold) with an attack that does 5 cold damage at initiative 18
582 units each with 3649 hit points with an attack that does 46 slashing damage at initiative 13
53 units each with 5147 hit points (immune to bludgeoning, slashing) with an attack that does 828 cold damage at initiative 11
5231 units each with 8051 hit points (weak to radiation) with an attack that does 14 radiation damage at initiative 9
704 units each with 4351 hit points (immune to cold; weak to slashing) with an attack that does 60 radiation damage at initiative 2
326 units each with 9157 hit points (weak to cold, slashing) with an attack that does 261 radiation damage at initiative 6
6980 units each with 3363 hit points (weak to radiation) with an attack that does 4 slashing damage at initiative 4

Infection:
1994 units each with 48414 hit points (immune to slashing) with an attack that does 46 cold damage at initiative 3
42 units each with 41601 hit points (weak to radiation; immune to fire) with an attack that does 1547 bludgeoning damage at initiative 7
3050 units each with 29546 hit points (immune to fire) with an attack that does 19 fire damage at initiative 10
3825 units each with 5609 hit points (immune to cold; weak to slashing, bludgeoning) with an attack that does 2 bludgeoning damage at initiative 12
37 units each with 30072 hit points with an attack that does 1365 cold damage at initiative 20
189 units each with 49726 hit points (weak to bludgeoning) with an attack that does 514 slashing damage at initiative 5
930 units each with 39623 hit points (weak to radiation) with an attack that does 81 bludgeoning damage at initiative 8
6343 units each with 31638 hit points (immune to slashing) with an attack that does 9 bludgeoning damage at initiative 15
1561 units each with 10633 hit points (weak to radiation, cold) with an attack that does 12 cold damage at initiative 14
3198 units each with 25539 hit points (immune to radiation, fire) with an attack that does 15 bludgeoning damage at initiative 17
""".strip()

def doit(boost=0, part1=False):
    lines = inp.splitlines()
    immune, infection = inp.split("\n\n")

    teams = []

    REGEX = re.compile(r"(\d+) units each with (\d+) hit points (\([^)]*\) )?with an attack that does (\d+) (\w+) damage at initiative (\d+)")

    # namedtuple? who needs namedtuple with hacks like these?
    UNITS, HP, DAMAGE, DTYPE, FAST, IMMUNE, WEAK = range(7)

    blah = boost
    for inps in [immune, infection]:
        lines = inps.splitlines()[1:]
        team = []
        for line in lines:
            s = REGEX.match(line)
            units, hp, extra, damage, dtype, fast = s.groups()
            immune = []
            weak = []
            if extra:
                extra = extra.rstrip(" )").lstrip("(")
                for s in extra.split("; "):
                    if s.startswith("weak to "):
                        weak = s[len("weak to "):].split(", ")
                    elif s.startswith("immune to "):
                        immune = s[len("immune to "):].split(", ")
                    else:
                        assert False
            u = [int(units), int(hp), int(damage) + blah, dtype, int(fast), set(immune), set(weak)]
            team.append(u)
        teams.append(team)
        blah = 0
    
    def power(t):
        return t[UNITS] * t[DAMAGE]

    def damage(attacking, defending):
        mod = 1
        if attacking[DTYPE] in defending[IMMUNE]:
            mod = 0
        elif attacking[DTYPE] in defending[WEAK]:
            mod = 2
        return power(attacking) * mod
    
    def sort_key(attacking, defending):
        return (damage(attacking, defending), power(defending), defending[FAST])
    
    while all(not all(u[UNITS] <= 0 for u in team) for team in teams):
        teams[0].sort(key=power, reverse=True)
        teams[1].sort(key=power, reverse=True)

        targets = []

        # target selection
        for team_i in range(2):
            other_team_i = 1 - team_i
            team = teams[team_i]
            other_team = teams[other_team_i]

            remaining_targets = set(i for i in range(len(other_team)) if other_team[i][UNITS] > 0)
            my_targets = [None] * len(team)

            for i, t in enumerate(team):
                if not remaining_targets:
                    break
                best_target = max(remaining_targets, key= lambda i: sort_key(t, other_team[i]))
                enemy = other_team[best_target]
                if damage(t, enemy) == 0:
                    continue
                my_targets[i] = best_target
                remaining_targets.remove(best_target)
            targets.append(my_targets)
        
        # attacking
        attack_sequence = [(0, i) for i in range(len(teams[0]))] + [(1, i) for i in range(len(teams[1]))]
        attack_sequence.sort(key=lambda x: teams[x[0]][x[1]][FAST], reverse=True)
        did_damage = False
        for team_i, index in attack_sequence:
            to_attack = targets[team_i][index]
            if to_attack is None:
                continue
            me = teams[team_i][index]
            other = teams[1-team_i][to_attack]

            d = damage(me, other)
            d //= other[HP]

            if teams[1-team_i][to_attack][UNITS] > 0 and d > 0:
                did_damage = True

            teams[1-team_i][to_attack][UNITS] -= d
            teams[1-team_i][to_attack][UNITS] = max(teams[1-team_i][to_attack][UNITS], 0)
        if not did_damage:
            return None
    
    if part1:
        return sum(u[UNITS] for u in teams[0]) or sum(u[UNITS] for u in teams[1])
    asd = sum(u[UNITS] for u in teams[0])
    if asd == 0:
        return None
    else:
        return asd
print(doit(part1=True))
# I did a manual binary search, submitted the right answer, then added in did_damage.
# Turns out that doit can infinite loop without the did_damage check!
# WARNING: "doit" is not guaranteed to be monotonic! You should manually check values yourself.
# print(doit(33))
maybe = binary_search(doit)
print(doit(maybe))

14799
4428
